<a href="https://colab.research.google.com/github/abhinavnautiyalDS/Finwise-GenAI-Assistant/blob/main/finwise-genai-capstone/task-02-agent-tools-langgraph/Task_2%20Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing required packages

In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
!pip install -qU langchain langchain-google-genai langgraph langchain-community langchain-experimental requests duckduckgo-search ddgs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

Importing Required Libraries

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_experimental.utilities.python import PythonREPL
from langchain_community.tools import DuckDuckGoSearchRun
from google.colab import userdata
import math
import requests
import json

Getting Api key from colab secrets

In [ ]:

google_api_key = userdata.get('GOOGLE_API_KEY')
alpha_vantage_key = userdata.get('ALPHA_VANTAGE_KEY')  # Optional for stock tool

# Initialize Gemini LLM (gemini-2.5-pro for advanced reasoning)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=google_api_key,
    temperature=0.1  # Low for precise calculations
)


Defining Tools

In [ ]:

@tool(return_direct=False)
def calculator(expression: str) -> str:
    """Perform mathematical calculations or EMI. Input:
    - 'EMI(P, r, n)' for loan EMI where P=principal, r=annual rate/12 (e.g., 0.09 for 9%), n=months.
    - Simple expressions like '2 + 2'.
    Returns result or error message."""
    try:
        if expression.startswith("EMI("):
            args = expression[4:-1].split(",")
            P = float(args[0].strip())
            r_annual = float(args[1].strip())
            r = r_annual / 12  # Monthly rate
            n = int(args[2].strip())
            emi = P * r * (math.pow(1 + r, n)) / (math.pow(1 + r, n) - 1)
            return f"Monthly EMI: ₹{emi:.2f}"
        else:
            return str(eval(expression))
    except Exception as e:
        return f"Error in calculation: {str(e)}"

@tool(return_direct=False)
def python_repl(code: str) -> str:
    """Run Python code for complex calculations or simulations. Input: valid Python code string."""
    try:
        repl = PythonREPL()
        result = repl.run(code)
        return str(result)
    except Exception as e:
        return f"REPL error: {str(e)}"

@tool(return_direct=False)
def stock_price(symbol: str) -> str:
    """Get current stock price for a symbol (e.g., AAPL). Returns price or error."""
    if not alpha_vantage_key:
        return "Alpha Vantage API key not set. Simulated price: $150.00 for AAPL."
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={alpha_vantage_key}"
    try:
        response = requests.get(url)
        data = response.json()
        price = float(data['Global Quote']['05. price'])
        return f"Current price for {symbol}: ${price:.2f}"
    except Exception as e:
        return f"Error fetching {symbol} data: {str(e)}"

@tool(return_direct=False)
def web_search(query: str) -> str:
    """Search the web for general information. Input: search query."""
    try:
        search = DuckDuckGoSearchRun()
        return search.run(query)
    except Exception as e:
        return f"Search error: {str(e)}"

# List of tools
tools = [calculator, python_repl, stock_price, web_search]

# Bind tools to LLM for better calling (fixes search/REPL issues)
llm_with_tools = llm.bind_tools(tools)

# Refined System Prompt with strict tool calling instructions
system_prompt = """You are a precise financial assistant. ALWAYS use tools for calculations and data fetching. Think step-by-step: 1. Identify the tool needed. 2. Call the tool with exact input. 3. Use the observation to form the answer.
- For EMI: Call 'calculator' with 'EMI(2000000, 0.09, 60)' (r=annual rate 0.09, n=months 60).
- For compound interest: Call 'python_repl' with code 'P = 10000; r = 0.07; t = 10; amount = P * (1 + r)**t; interest = amount - P; print(f"Interest: {interest:.2f}")'.
- For stock prices: Call 'stock_price' with symbol 'AAPL'.
- For general info: Call 'web_search' with query.
Reference prior observations in multi-step queries. End with clear final answer. No disclaimers unless asked."""

Creating Agents

In [ ]:
# Create ReAct Agent with LangGraph (latest method, using bound LLM)
agent = create_react_agent(
    llm_with_tools,  # Bound LLM
    tools,  # Tools list
    # messages_modifier=system_prompt  # Refined prompt
)

Testing

In [ ]:

print("=== Testing LangGraph ReAct Agent with External Tools ===")

# Test 1: EMI Calculation
query1 = "What is the monthly EMI for a 20 lakh loan over 5 years at 9% annual interest?"
print(f"\nQuery 1: {query1}")
response1 = agent.invoke({"messages": [("human", query1)]})
print("Final Response:")
print(response1["messages"][-1].content)  # Last AI message
print("\nFull Trajectory (Tool Calls):")
for msg in response1["messages"]:
    if msg.type == "tool":
        print(f"Tool: {msg.name} - {msg.content}")

# Test 2: Stock Price
query2 = "What's the current price of AAPL stock, and how much would 10 shares cost?"
print(f"\nQuery 2: {query2}")
response2 = agent.invoke({"messages": [("human", query2)]})
print("Final Response:")
print(response2["messages"][-1].content)
print("\nFull Trajectory (Tool Calls):")
for msg in response2["messages"]:
    if msg.type == "tool":
        print(f"Tool: {msg.name} - {msg.content}")

# Test 3: Compound Interest with Python REPL
query3 = "Calculate compound interest on 10,000 at 7% annual rate for 10 years."
print(f"\nQuery 3: {query3}")
response3 = agent.invoke({"messages": [("human", query3)]})
print("Final Response:")
print(response3["messages"][-1].content)
print("\nFull Trajectory (Tool Calls):")
for msg in response3["messages"]:
    if msg.type == "tool":
        print(f"Tool: {msg.name} - {msg.content}")

# Multi-Step Test (Context-Aware)
print("\n=== Multi-Step Test (Context-Aware) ===")
# Chain previous response with new query
messages_multi = response2["messages"] + [("human", "Using the AAPL price above, calculate total cost for 10 shares.")]
response_multi = agent.invoke({"messages": messages_multi})
print("Final Response:")
print(response_multi["messages"][-1].content)
print("\nFull Trajectory (Tool Calls):")
for msg in response_multi["messages"]:
    if msg.type == "tool":
        print(f"Tool: {msg.name} - {msg.content}")

# Test 4: Web Search (to verify search calling)
print("\n=== Test 4: Web Search ===")
query4 = "What is the current inflation rate in India?"
response4 = agent.invoke({"messages": [("human", query4)]})
print("Final Response:")
print(response4["messages"][-1].content)
print("\nFull Trajectory (Tool Calls):")
for msg in response4["messages"]:
    if msg.type == "tool":
        print(f"Tool: {msg.name} - {msg.content}")

=== Testing LangGraph ReAct Agent with External Tools ===

Query 1: What is the monthly EMI for a 20 lakh loan over 5 years at 9% annual interest?
Final Response:
The monthly EMI for a 20 lakh loan over 5 years at 9% annual interest is ₹41516.71.

Full Trajectory (Tool Calls):
Tool: calculator - Monthly EMI: ₹41516.71

Query 2: What's the current price of AAPL stock, and how much would 10 shares cost?
Final Response:
The current price of AAPL is $256.69. Therefore, 10 shares would cost $2566.90.

Full Trajectory (Tool Calls):
Tool: stock_price - Current price for AAPL: $256.69

Query 3: Calculate compound interest on 10,000 at 7% annual rate for 10 years.


Final Response:
The compound interest is approximately 9671.51.

Full Trajectory (Tool Calls):
Tool: python_repl - 9671.513572895663


=== Multi-Step Test (Context-Aware) ===
Final Response:
The total cost for 10 shares of AAPL at the current price of $256.69 is $2566.90.

Full Trajectory (Tool Calls):
Tool: stock_price - Current price for AAPL: $256.69
Tool: calculator - 2566.9

=== Test 4: Web Search ===
Final Response:
I am sorry, I cannot fulfill this request. I encountered an error when trying to search the web for the current inflation rate in India.

Full Trajectory (Tool Calls):
Tool: web_search - Search error: Could not import ddgs python package. Please install it with `pip install -U ddgs`.
